# Ejercicios prácticos - Módulo 2

In [1]:
import arviz as az
import pymc as pm
import preliz as pz
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


### 1. Usando PyMC reproducí los resultados del primer capítulo para el problema de la moneda. Usá los 3 priors usados en ese capítulo Beta(1, 1), Beta(0.5, 0.5), Beta(20, 20). Probá con 4 experimentos y con 150. Presenta los resultados usando 2 visualizaciones.

### 2. En `nuestro_primer_modelo` reemplazá la distribución $\theta \sim Beta(1, 1)$ por una uniforme en el intervalo [0, 1] Cual es el efecto en el muestreo y en la distribución a posteriori? ¿Y si usas un intervalo más ámplio, como [-2, 1]?

### 3. Los modelos de PyMC tienen un método `.to_graphviz()` que se puede utilizar para visualizar el modelo (también existe la función `pm.model_to_graphviz()`. Use este método para visualizar `nuestro_primer_modelo`. Compare el resultado con el diagrama de Kruschke.

### 4. En el modelo `comparación_grupos` que dimensión y tamaño tiene μ[idx] y σ[idx]? Justifique

### 5. La Gaussiana es una distribución sin límites es decir es válida en el intervalo $(-\infty, \infty)$, en el modelo `comparación_grupos` usamos una Gama como likelihood, pero podríamos haber usado una Gaussiana? Cambiarías tu respuesta si las propinas tomaran valores alrededor de 100 dolares?

### 6. Calculá la probabilidad de superioridad directamente a partir de las muestras de la distribución *a posteriori*, es decir sin la expresión analítica. Cómo se comparan los resultados con los valores obtenidos analíticamente. 

### 7. Aplicá al menos uno de los modelos vistos en este capítulo a datos propios o de tu interés.